In [11]:
import pandas as pd
import numpy as np

In [17]:
data_set = pd.read_csv('btc_data.csv')

In [19]:
data_set.head()

,Open,High,Low,Adj Close,mean,RSI,EMAF,EMAM,EMAS,Target,TargetClass,TargetNextClose
0,10123.035156,10215.948242,9980.776367,10178.372070,0.1725,47.342426,10294.158326,9692.104233,9123.295244,233.307617,1,10410.126953
1,10176.819336,10442.253906,10099.242188,10410.126953,0.2258,51.280170,10305.202957,9706.322505,9140.339373,-54.815430,0,10360.546875
2,10415.362305,10441.489258,10226.596680,10360.546875,0.0590,50.416005,10310.473807,9719.277443,9156.501062,12.645508,1,10358.048828
3,10345.403320,10422.133789,10291.694336,10358.048828,0.2153,50.370183,10315.004761,9731.926381,9172.415601,-8.752930,0,10347.712891
4,10356.465820,10387.035156,10313.092773,10347.712891,0.1545,50.168045,10318.119821,9744.120174,9187.982453,-70.428711,0,10276.793945


In [18]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.08084276 0.076507   0.09432016 ... 0.52423667 1.         0.08584866]
 [0.08169191 0.08005195 0.09622245 ... 0.50486527 0.         0.08506614]
 [0.08545807 0.08003998 0.09826749 ... 0.50940088 1.         0.08502671]
 ...
 [0.91783745 0.9900751  0.93584042 ... 0.20332226 0.         0.92851576]
 [1.         1.         0.88666169 ... 0.66525442 1.         0.96490495]
 [0.92792684 0.97599116 0.94326157 ... 0.56406786 1.         0.97782611]]


In [22]:
# multiple feature from data provided to the model
X = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 30
print(data_set_scaled.shape[0])
for j in range(9):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X)
print(X.shape)
print(y)
print(y.shape)

1639
[[[0.08084276 0.076507   0.09432016 ... 0.06648817 0.04521002 0.03021636]
  [0.08169191 0.08005195 0.09622245 ... 0.06668302 0.04550981 0.03059896]
  [0.08545807 0.08003998 0.09826749 ... 0.06677601 0.04578297 0.03096175]
  ...
  [0.05122118 0.04700717 0.06549622 ... 0.03783141 0.03903427 0.02961549]
  [0.05095264 0.05162805 0.06523184 ... 0.03770696 0.0386988  0.0294666 ]
  [0.05656439 0.05158993 0.07009148 ... 0.03757879 0.03836611 0.02931692]]

 [[0.08169191 0.08005195 0.09622245 ... 0.06668302 0.04550981 0.03059896]
  [0.08545807 0.08003998 0.09826749 ... 0.06677601 0.04578297 0.03096175]
  [0.08435355 0.07973678 0.09931281 ... 0.06685594 0.04604967 0.03131899]
  ...
  [0.05095264 0.05162805 0.06523184 ... 0.03770696 0.0386988  0.0294666 ]
  [0.05656439 0.05158993 0.07009148 ... 0.03757879 0.03836611 0.02931692]
  [0.05656411 0.05310167 0.06759045 ... 0.03701805 0.03792947 0.02909052]]

 [[0.08545807 0.08003998 0.09826749 ... 0.06677601 0.04578297 0.03096175]
  [0.08435355 0.0

In [24]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

1287
(1287, 30, 9)
(322, 30, 9)
(1287, 1)
(322, 1)
[[0.05312165]
 [0.05287622]
 [0.05372346]
 ...
 [0.40130863]
 [0.37645253]
 [0.36735066]]


In [26]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(backcandles, 9), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_6812\2963505729.py", line 1, in <module>
    from keras.models import Sequential
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\__init__.py", line 3, in <module>
    from keras import __internal__
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\__internal__\__init__.py", line 6, in <module>
    from keras.__internal__ import models
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\__internal__\models\__init__.py", line 3, in <module>
    from keras.src.models.cloning import clone_and_build_model
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\__init__.py", line 21, in <module>
    from keras.src import applications
  File "C:\Users\

In [ ]:
y_pred = model.predict(X_test)
#y_pred=np.where(y_pred > 0.43, 1,0)
for i in range(10):
    print(y_pred[i], y_test[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test, color = 'black', label = 'Test')
plt.plot(y_pred, color = 'green', label = 'pred')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import fabs

def evaluate_model(y_test, y_pred):
  """
  This function calculates various metrics to evaluate the LSTM model's performance.

  Args:
      y_test (list): List of actual target values.
      y_pred (list): List of predicted target values.

  Returns:
      dict: Dictionary containing the calculated metrics (MAE, MSE, R-squared, SMAPE, KGE).
  """
  # Mean Absolute Error (MAE)
  mae = mean_absolute_error(y_test, y_pred)

  # Mean Squared Error (MSE)
  mse = mean_squared_error(y_test, y_pred)

  # R-squared
  r2 = r2_score(y_test, y_pred)

  # Symmetric Mean Absolute Percentage Error (SMAPE)
  smape = 0
  for i in range(len(y_test)):
    if y_test[i] == 0:
      continue  # Avoid division by zero
    smape += fabs((y_pred[i] - y_test[i]) / (abs(y_test[i]) + abs(y_pred[i]))) / len(y_test)
  smape *= 2 * 100  # Convert to percentage

  # Kling-Gupta Efficiency (KGE)
  def mean(arr):
    return sum(arr) / len(arr)

  y_bar = mean(y_test)
  sigma_y = np.std(y_test)
  sigma_p = np.std(y_pred)
  r = np.corrcoef(y_test, y_pred)[0, 1]
  alpha = 1  # Weighting factor for mean relative error (can be adjusted)
  kge = 1 - np.sqrt( 
      (mean((y_test - y_pred) ** 2) / (sigma_y ** 2)) + 
      (fabs(mean(y_test) - mean(y_pred)) / y_bar) ** alpha + 
      (fabs(sigma_y - sigma_p) / sigma_y) ** 2
  )

  return {
      "MAE": mae,
      "MSE": mse,
      "R-squared": r2,
      "SMAPE": smape,
      "KGE": kge
  }

# Call the function after your model prediction
metrics = evaluate_model(y_test, y_pred)
print(metrics)